<a href="https://colab.research.google.com/github/herysedra/covid19-mankaiza-clone/blob/andrana/scripts/paper/simple_blocks/simple_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pymc3 as pm
import theano.tensor as tt
import theano
import datetime
import time

date_data_begin = datetime.date(2020,3,1)
date_data_end = datetime.date(2020,3,15)


diff_data_sim = 16

date_begin_sim = date_data_begin - datetime.timedelta(days = diff_data_sim)
num_days = (date_data_end - date_begin_sim).days


print(date_begin_sim)
print(num_days)

2020-02-14
30


theano.tensor.var.TensorVariable

In [15]:
tt.ones(num_days-1)

Alloc.0

In [0]:
print(tt.ones(num_days-1))

In [16]:
type(tt.ones(num_days-1))

theano.tensor.var.TensorVariable

In [17]:
tt.ones(num_days-1).shape

Shape.0

In [19]:
2*tt.ones(num_days-1)

Elemwise{mul,no_inplace}.0

In [21]:
print(2*tt.ones(num_days-1))

Elemwise{mul,no_inplace}.0


In [22]:
type(2*tt.ones(num_days-1))

theano.tensor.var.TensorVariable

In [23]:
2*tt.ones(num_days-1).shape

Elemwise{mul,no_inplace}.0